In [1]:
import requests
import json
import psycopg2
import psycopg2.extras
from datetime import datetime
import numpy as np
import calendar
from pandas import DataFrame
import pandas as pd
import time

In [ ]:
url = 'http://api.wunderground.com/api/9112e570d64832fa/history_20170504/q/Thailand/Chiang_rai.json'
data_hist = requests.get(url).json()
data_day = data_hist['history']['observations']
len_per_day = len(data_day)
# print(data_hist['history']['observations'][0]['date']['pretty'])
# print(data_hist['history']['observations'][0]['tempm'])
# print(data_hist['history']['observations'][0]['hum'])
# print(data_hist['history']['observations'][0]['conds'])
# print(data_hist['history']['observations'][0]['icon'])
# print(data_hist['history']['observations'][0]['fog'])
# print(data_hist['history']['observations'][0]['rain'])
# print(data_hist['history']['observations'][0]['thunder'])

In [ ]:
# read activepower from csv
df_load = pd.read_csv("load_power.csv",parse_dates = True, index_col = 0)
# print(df_load[:5])
# df.index = pd.to_datetime(df['datetime'])
# df_avg = df.resample('15min').mean()

# print(df_load.index[0])
# print(type(df_load.index[0]))
# print(df_load['activepower'][0])
# print(str(df_load['activepower'][0]))
# print(len(df_load))

In [2]:
# database
db_host = "peahivestaging.postgres.database.azure.com"
db_port = "5432"
db_database = "postgres"
db_user = "peahive@peahivestaging"
db_password = "28Sep1960"

In [ ]:
# connects to database
try:
    con = psycopg2.connect(host=db_host, port=db_port, database=db_database, user=db_user, password=db_password)
    cur = con.cursor()
    print ("Connects to database successfully")
except Exception as er:
    print ("CONNECTION ERROR: {}".format(er))

In [ ]:
# activepower in database
for i in range(750):
    try:
        query = "INSERT INTO ts_feature_load (datetime, power) " + "VALUES ('"+df_load.index[i]+"','"+str(df_load['activepower'][i])+"')"
        #print(query)
        cur.execute(query)
        con.commit()
    except Exception as er:
        print ("error: {}".format(er))

In [ ]:
# disconnects to database
try:
    if (con.closed == False):
        con.close()
        print ("Disconnects to database successfully")
except Exception as er:
    print ("DISCONNECTION ERROR: {}".format(er))

In [ ]:
try:
    con = psycopg2.connect(host=db_host, port=db_port, database=db_database, user=db_user, password=db_password)
    cur = con.cursor()
    print ("Connects to database successfully")
except Exception as er:
    print ("CONNECTION ERROR: {}".format(er))
    
url = 'http://api.wunderground.com/api/9112e570d64832fa/history_20170531/q/Thailand/Chiang_rai.json'
data_hist = requests.get(url).json()
data_day = data_hist['history']['observations']
len_per_day = len(data_day)

df = {}
for i in range(len_per_day):
#     format1 = '%Y-%m-%d %H:%M:%S+00:00'
#     temp_time = str(datetime.strptime(str(df_time['datetime'][i]), format1))
#     print(temp_time)
    web_time = str(datetime.strptime(str(data_hist['history']['observations'][i]['date']['pretty']), '%I:%M %p +07 on %B %d, %Y'))
    #print(web_time)
    df['pretty'] = web_time
    df['tempm']  = data_hist['history']['observations'][i]['tempm']
    if (data_hist['history']['observations'][i]['hum'] == 'N/A'):
        df['hum']  = str(-9999)
    else:
        df['hum']  = data_hist['history']['observations'][i]['hum']
        
    df['winds']  = data_hist['history']['observations'][i]['windchillm']
    df['pressure']  = data_hist['history']['observations'][i]['pressurem']
    df['heat_index']  = data_hist['history']['observations'][i]['heatindexm']
    df['conds']  = data_hist['history']['observations'][i]['conds']
    df['icon']  = data_hist['history']['observations'][i]['icon']
    df['fog']  = data_hist['history']['observations'][i]['fog']
    df['rain']  = data_hist['history']['observations'][i]['rain']
    df['snow']  = data_hist['history']['observations'][i]['snow']
    df['hail']  = data_hist['history']['observations'][i]['hail']
    df['thunder']  = data_hist['history']['observations'][i]['thunder']
    df['tornado']  = data_hist['history']['observations'][i]['tornado']

    try:
        query = "UPDATE ts_feature_load SET tempc='"+df['tempm']+"', humidity='"+df['hum']+"', wind_speed='"+df['winds']+"', pressure='"+df['pressure']+"', heat_index='"+df['heat_index']+"', cond='"+df['conds']+"', icon='"+df['icon']+"', fog='"+df['fog']+"', rain='"+df['rain']+"', snow='"+df['snow']+"', hail='"+df['hail']+"', thunder='"+df['thunder']+"', tornado='"+df['tornado']+"' WHERE datetime= '"+web_time+"'"
        #print(query)
        cur.execute(query)
        con.commit()
    except Exception as er:
        print ("error: {}".format(er))
        
try:
    if (con.closed == False):
        con.close()
        print ("Disconnects to database successfully")
except Exception as er:
    print ("DISCONNECTION ERROR: {}".format(er))
    

In [3]:
# select database
try:
    con = psycopg2.connect(host=db_host, port=db_port, database=db_database, user=db_user, password=db_password)
    cur = con.cursor()
    print ("Connects to database successfully")
except Exception as er:
    print ("CONNECTION ERROR: {}".format(er))
    
query_df = "SELECT * FROM ts_feature_load ORDER BY datetime"
df_all = pd.read_sql_query(query_df, con)

for i in range(len(df_all)):
    if (np.isnan(df_all['tempc'][i]) == True):
        df_all['tempc'][i] = '25'
    if (df_all['tempc'][i] == -9999):
        df_all['tempc'][i] = '25'
        print('OK')
        
for i in range(len(df_all)):
    if (np.isnan(df_all['humidity'][i]) == True):
        df_all['humidity'][i] = '65'
    if (df_all['humidity'][i] == -9999):
        df_all['humidity'][i] = '65'
        print('OK')

try:
    if (con.closed == False):
        con.close()
        print ("Disconnects to database successfully")
except Exception as er:
    print ("DISCONNECTION ERROR: {}".format(er))
    



Connects to database successfully


C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


OK
OK
OK


C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


OK
OK
OK
Disconnects to database successfully


In [4]:
df_test = df_all[['datetime', 'tempc', 'humidity', 'power']].copy()
print(df_test.head())

                   datetime  tempc  humidity      power
0 2017-05-01 00:00:00+00:00   25.0      94.0  19.500000
1 2017-05-01 01:00:00+00:00   24.0      94.0  18.166667
2 2017-05-01 02:00:00+00:00   25.0      65.0  14.666667
3 2017-05-01 03:00:00+00:00   25.0      65.0  14.000000
4 2017-05-01 04:00:00+00:00   23.0     100.0  13.333333


In [5]:
for i in range(24):
    keys_hour = 'H'+str(i)
    df_test[keys_hour]= 0

for i in range(len(df_test)):
    temp_time = datetime.strptime(str(df_test['datetime'][i]), '%Y-%m-%d %H:%M:%S+00:00')
    temp_hour = temp_time.hour
    #print(temp_hour)
    keys_hour = 'H'+str(temp_hour)
    #print(keys_hour)
    
    # column HX
    df_test[keys_hour][i] = 1
        
print(df_test.columns)
print(df_test.head())

C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Phanumat\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

Index(['datetime', 'tempc', 'humidity', 'power', 'HH', 'H0', 'H1', 'H2', 'H3',
       'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14',
       'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'Night',
       'Morning', 'Afternoon', 'Evening'],
      dtype='object')
                   datetime  tempc  humidity      power  HH  H0  H1  H2  H3  \
0 2017-05-01 00:00:00+00:00   25.0      94.0  19.500000   0   1   0   0   0   
1 2017-05-01 01:00:00+00:00   24.0      94.0  18.166667   1   0   1   0   0   
2 2017-05-01 02:00:00+00:00   25.0      65.0  14.666667   2   0   0   1   0   
3 2017-05-01 03:00:00+00:00   25.0      65.0  14.000000   3   0   0   0   1   
4 2017-05-01 04:00:00+00:00   23.0     100.0  13.333333   4   0   0   0   0   

   H4   ...     H18  H19  H20  H21  H22  H23  Night  Morning  Afternoon  \
0   0   ...       0    0    0    0    0    0      1        0          0   
1   0   ...       0    0    0    0    0    0      1        0          0   

In [6]:
print(df_all.head())
print(df_test)

                   datetime  tempc  humidity  wind_speed  pressure  \
0 2017-05-01 00:00:00+00:00   25.0      94.0      -999.0    1010.0   
1 2017-05-01 01:00:00+00:00   24.0      94.0      -999.0    1010.0   
2 2017-05-01 02:00:00+00:00   25.0      65.0         NaN       NaN   
3 2017-05-01 03:00:00+00:00   25.0      65.0         NaN       NaN   
4 2017-05-01 04:00:00+00:00   23.0     100.0      -999.0    1010.0   

   heat_index   cond   icon  fog  rain  snow  hail  thunder  tornado  \
0     -9999.0  Clear  clear  0.0   0.0   0.0   0.0      0.0      0.0   
1     -9999.0  Clear  clear  0.0   0.0   0.0   0.0      0.0      0.0   
2         NaN   None   None  NaN   NaN   NaN   NaN      NaN      NaN   
3         NaN   None   None  NaN   NaN   NaN   NaN      NaN      NaN   
4     -9999.0  Clear  clear  0.0   0.0   0.0   0.0      0.0      0.0   

       power  
0  19.500000  
1  18.166667  
2  14.666667  
3  14.000000  
4  13.333333  
                     datetime  tempc  humidity      powe

In [7]:
try:
    df_test.to_csv('load_dataset.csv', encoding='utf-8', index=False)
except:
    print('error')

# 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
%matplotlib inline

# 2. Import Data

In [ ]:
# file name: df_test
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.describe()

In [ ]:
df_test.columns

# 3.Data Visualization & Data Cleaning

In [ ]:
sns.pairplot(df_test)

In [ ]:
sns.distplot(df_test['power'])

In [ ]:
sns.heatmap(df_test.corr(), annot = True)

# 4. X: feature, y: target

In [8]:
X = df_test[['tempc', 'humidity', 'HH', 'H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'Night', 'Morning', 'Evening']]
y = df_test['power']

# 5. Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# 6. Model Training

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

# 7. Model Evalution

In [ ]:
# print the intercept
print(lm.intercept_)

In [ ]:
# print the coefficient
print(lm.coef_)

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df

# 8. Make Predictions From the Model

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
py.iplot{[]}

In [ ]:
sns.distplot((y_test-predictions),bins=50);

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))